<a href="https://colab.research.google.com/github/buddypia/openai-agents-sdk-colab/blob/master/simple_mcp_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MCP活用エージェント

MCPサーバーの作成エージェントでそのMCPサーバーを活用する例
1. MCPサーバーの作成
2. addツール、get_secret_word、get_current_weatherツールを定義
3. MCPサーバーへ接続
4. 各ツールの確認
5. エージェントでMCPサーバーの利用

## INPUT1
```markdown
これらの数字を足す: 7 and 22.
```

## OUTPUT1
```markdown
7と22を足すと、29です。
```

## INPUT2
```markdown
東京の天気はどうですか？
```

## OUTPUT2
```markdown
東京の現在の天気は「**晴れ時々曇り**」です。気温は約20°C (68°F)で、北北西からの風が約21 km/h (13 mph)で吹いています。降水の可能性は低く、視界は約9.7 km (6マイル)です。

午後や夕方には一時的に雨が降る可能性があります。気温の変動に注意してください。
```

## INPUT3
```markdown
秘密の言葉は何ですか？
```

## OUTPUT3
```markdown
秘密の言葉は「りんご」です。
```

## エージェントフロー図
<img src="https://drive.google.com/uc?id=1L5gmVDenxiScUX9kfc4viUUBIMryrUIc" width="80%">

In [ ]:
# Pythonパッケージインストール
!pip install openai-agents "mcp[cli]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.1 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata

# ColabのシークレットからOpenAIのAPIキーを取得
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

# OpenAIエージェントにAPIキーを設定
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [ ]:
# MCPサーバーの実装
import random

import requests
from mcp.server.fastmcp import FastMCP # FastMCPサーバーフレームワーク

mcp = FastMCP("Echo Server") # "Echo Server"という名前でFastMCPサーバーインスタンスを作成

@mcp.tool()
def add(a: int, b: int) -> int:
    """二つの数を足し合わせる"""
    print(f"[debug-server] add({a}, {b})")
    return a + b


@mcp.tool()
def get_secret_word() -> str:
    """秘密の単語を取得する"""
    print("[debug-server] get_secret_word()")
    return random.choice(["りんご", "バナナ", "みかん"])


@mcp.tool()
def get_current_weather(city: str) -> str:
    """指定された都市の現在の天気を取得する"""
    print(f"[debug-server] get_current_weather({city})")

    endpoint = "https://wttr.in"
    response = requests.get(f"{endpoint}/{city}")
    return response.text

In [ ]:
import asyncio

# colabでMCPサーバーを起動
server_task = asyncio.create_task(mcp.run_sse_async())
# ローカル環境でMCPサーバーを起動する場合はこちらを利用
# asyncio.run(mcp.run_sse_async())

In [ ]:
# MCPサーバーに接続
from agents.mcp import MCPServerSse

mcp_server = MCPServerSse(params={"url": "http://0.0.0.0:8000/sse"})
await mcp_server.connect()

INFO:     127.0.0.1:54094 - "GET /sse HTTP/1.1" 200 OK
INFO:     127.0.0.1:54110 - "POST /messages/?session_id=05a71fcc7d0d49439ff0f0870bbd80fe HTTP/1.1" 202 Accepted


In [ ]:
# サーバー上で使用可能なツールを一覧表示
await mcp_server.list_tools()

INFO:     127.0.0.1:54110 - "POST /messages/?session_id=05a71fcc7d0d49439ff0f0870bbd80fe HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:54110 - "POST /messages/?session_id=05a71fcc7d0d49439ff0f0870bbd80fe HTTP/1.1" 202 Accepted


[Tool(name='add', description='二つの数を足し合わせる', inputSchema={'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'addArguments', 'type': 'object'}),
 Tool(name='get_secret_word', description='秘密の単語を取得する', inputSchema={'properties': {}, 'title': 'get_secret_wordArguments', 'type': 'object'}),
 Tool(name='get_current_weather', description='指定された都市の現在の天気を取得する', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_current_weatherArguments', 'type': 'object'})]

In [ ]:
# 二つの数を足し合わせる
await mcp_server.call_tool("add", {"a": 3, "b": 2})

INFO:     127.0.0.1:54110 - "POST /messages/?session_id=05a71fcc7d0d49439ff0f0870bbd80fe HTTP/1.1" 202 Accepted
[debug-server] add(3, 2)


CallToolResult(meta=None, content=[TextContent(type='text', text='5', annotations=None)], isError=False)

In [ ]:
# 秘密の単語を取得する
await mcp_server.call_tool("get_secret_word", {})

INFO:     127.0.0.1:41044 - "POST /messages/?session_id=05a71fcc7d0d49439ff0f0870bbd80fe HTTP/1.1" 202 Accepted
[debug-server] get_secret_word()


CallToolResult(meta=None, content=[TextContent(type='text', text='りんご', annotations=None)], isError=False)

In [ ]:
# 指定された都市の現在の天気を取得する
# https://wttr.in/shibuya
await mcp_server.call_tool("get_current_weather", {"city": "shibuya"})


INFO:     127.0.0.1:54110 - "POST /messages/?session_id=05a71fcc7d0d49439ff0f0870bbd80fe HTTP/1.1" 202 Accepted
[debug-server] get_current_weather(shibuya)


CallToolResult(meta=None, content=[TextContent(type='text', text='Weather report: shibuya\n\n  \x1b   \\  /\x1b       Partly cloudy\n  \x1b _ /""\x1b.-.    \x1b \x1b+71\x1b(\x1b75\x1b) °F\x1b     \n  \x1b   \\_\x1b(   ).  \x1b \x1b↑\x1b \x1b19\x1b mph\x1b       \n  \x1b   /\x1b(___(__) \x1b 6 mi\x1b           \n                0.0 in\x1b         \n                                                       ┌─────────────┐                                                       \n┌──────────────────────────────┬───────────────────────┤  Thu 01 May ├───────────────────────┬──────────────────────────────┐\n│            Morning           │             Noon      └──────┬──────┘     Evening           │             Night            │\n├──────────────────────────────┼──────────────────────────────┼──────────────────────────────┼──────────────────────────────┤\n│ \x1b    \\   /    \x1b Sunny          │ \x1b    \\   /    \x1b Sunny          │ \x1b    \\   /    \x1b Sunny          │ \x1b    \\   /    \x

In [ ]:
from agents import Agent
from agents.model_settings import ModelSettings

# ツールを使用するエージェントを定義
agent = Agent(
	name="Assistant",
	instructions="アシスタントです。ツールを使用して質問に答えます。",
	mcp_servers=[mcp_server],
	model_settings=ModelSettings(tool_choice="required") # ツールを必須にする
)

In [ ]:
from agents import Runner

# `add`ツールを使って2つの数字を足す。
message = "これらの数字を足す: 7 and 22."
print(f"実行中: {message}")
result = await Runner.run(starting_agent=agent, input=message)
print(result.final_output)

実行中: これらの数字を足す: 7 and 22.
INFO:     127.0.0.1:54110 - "POST /messages/?session_id=05a71fcc7d0d49439ff0f0870bbd80fe HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:54110 - "POST /messages/?session_id=05a71fcc7d0d49439ff0f0870bbd80fe HTTP/1.1" 202 Accepted
[debug-server] add(7, 22)
7と22を足すと、29です。


In [ ]:
# `get_weather`ツールを実行
message = "東京の天気はどうですか？"
print(f"実行中: {message}")
result = await Runner.run(starting_agent=agent, input=message)
print(result.final_output)

実行中: 東京の天気はどうですか？
INFO:     127.0.0.1:54110 - "POST /messages/?session_id=05a71fcc7d0d49439ff0f0870bbd80fe HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:54110 - "POST /messages/?session_id=05a71fcc7d0d49439ff0f0870bbd80fe HTTP/1.1" 202 Accepted
[debug-server] get_current_weather(Tokyo)
東京の現在の天気は「**晴れ時々曇り**」です。気温は約20°C (68°F)で、北北西からの風が約21 km/h (13 mph)で吹いています。降水の可能性は低く、視界は約9.7 km (6マイル)です。

午後や夕方には一時的に雨が降る可能性があります。気温の変動に注意してください。


In [ ]:
# `get_secret_word`ツールを実行
message = "秘密の言葉は何ですか？"
print(f"実行中: {message}")
result = await Runner.run(starting_agent=agent, input=message)
print(result.final_output)

実行中: 秘密の言葉は何ですか？
INFO:     127.0.0.1:41044 - "POST /messages/?session_id=05a71fcc7d0d49439ff0f0870bbd80fe HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:41044 - "POST /messages/?session_id=05a71fcc7d0d49439ff0f0870bbd80fe HTTP/1.1" 202 Accepted
[debug-server] get_secret_word()
秘密の言葉は「りんご」です。


In [ ]:
# colabのMCPサーバーを停止
server_task.cancel()

True